<a href="https://colab.research.google.com/github/ameralhomdy/DS-Unit-2-Regression-Classification/blob/master/module2/assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [6]:
import pandas_profiling
pandas_profiling.ProfileReport(df)

In [7]:
df.head()

BOROUGH             NEIGHBORHOOD  ... SALE_PRICE   SALE_DATE
0       1                    OTHER  ...          0  01/01/2019
1       1                    OTHER  ...          0  01/01/2019
2       1                    OTHER  ...          0  01/01/2019
3       1                    OTHER  ...          0  01/01/2019
4       1  UPPER EAST SIDE (59-79)  ...          0  01/01/2019

[5 rows x 21 columns]

In [0]:
# dropping Sale Price values under 100,000

df = df[(df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)]

In [0]:
# High NaN values

df = df.drop(columns=['EASE-MENT', 'APARTMENT_NUMBER'])

In [10]:
df.isnull().sum()

BOROUGH                            0
NEIGHBORHOOD                       0
BUILDING_CLASS_CATEGORY            0
TAX_CLASS_AT_PRESENT               0
BLOCK                              0
LOT                                0
BUILDING_CLASS_AT_PRESENT          0
ADDRESS                            0
ZIP_CODE                           0
RESIDENTIAL_UNITS                  0
COMMERCIAL_UNITS                   0
TOTAL_UNITS                        0
LAND_SQUARE_FEET                  37
GROSS_SQUARE_FEET                  0
YEAR_BUILT                        22
TAX_CLASS_AT_TIME_OF_SALE          0
BUILDING_CLASS_AT_TIME_OF_SALE     0
SALE_PRICE                         0
SALE_DATE                          0
dtype: int64

In [0]:
df = df.dropna()

In [0]:
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(',', '').astype(int)

In [0]:
df = df[df['LAND_SQUARE_FEET'] != 0]

In [0]:
# splitting the dataframe into test and train

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('2019-04-01')
train = df[df['SALE_DATE'] < cutoff]
test  = df[df['SALE_DATE'] >= cutoff]

In [0]:
# Excluding the non-numeric columns from the features with high cardinality for now

target = 'SALE_PRICE'
high_cardinality = ['ADDRESS', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_CATEGORY', 'SALE_DATE']
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
# OneHotEncoding

import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.fit_transform(X_test)

In [17]:
X_train.head()

BOROUGH_3  BOROUGH_4  ...  YEAR_BUILT  TAX_CLASS_AT_TIME_OF_SALE
44          1          0  ...      1930.0                          1
61          0          1  ...      1940.0                          1
74          0          0  ...      1930.0                          1
78          0          0  ...      1925.0                          1
79          0          0  ...      1935.0                          1

[5 rows x 35 columns]

In [18]:
X_train.describe()

BOROUGH_3    BOROUGH_4  ...   YEAR_BUILT  TAX_CLASS_AT_TIME_OF_SALE
count  6355.000000  6355.000000  ...  6355.000000                6355.000000
mean      0.273171     0.387254  ...  1879.456176                   1.259166
std       0.445623     0.487161  ...   360.841880                   0.649223
min       0.000000     0.000000  ...     0.000000                   1.000000
25%       0.000000     0.000000  ...  1920.000000                   1.000000
50%       0.000000     0.000000  ...  1935.000000                   1.000000
75%       1.000000     1.000000  ...  1965.000000                   1.000000
max       1.000000     1.000000  ...  2018.000000                   4.000000

[8 rows x 35 columns]

In [19]:
# feature selection with SelectKBest
from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k=15)

# IMPORTANT!
# .fit_transform on the train set
# .transform on test set

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

((6355, 15), (1695, 15))

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

1 features
Test MAE: $286,104 

2 features
Test MAE: $272,377 

3 features
Test MAE: $319,003 

4 features
Test MAE: $297,512 

5 features
Test MAE: $304,495 

6 features
Test MAE: $302,067 

7 features
Test MAE: $305,864 

8 features
Test MAE: $317,536 

9 features
Test MAE: $377,184 

10 features
Test MAE: $380,773 

11 features
Test MAE: $397,012 

12 features
Test MAE: $397,503 

13 features
Test MAE: $389,714 

14 features
Test MAE: $430,576 

15 features
Test MAE: $435,210 

16 features
Test MAE: $435,944 

17 features
Test MAE: $371,879 

18 features
Test MAE: $379,478 

19 features
Test MAE: $378,499 

20 features
Test MAE: $378,571 

21 features
Test MAE: $378,760 

22 features
Test MAE: $416,037 

23 features
Test MAE: $415,117 

24 features
Test MAE: $416,489 

25 features
Test MAE: $416,489 

26 features
Test MAE: $416,881 

27 features
Test MAE: $416,834 

28 features
Test MAE: $416,285 

29 features
Test MAE: $416,285 

30 features
Test MAE: $398,659 

31 features
Test MA

In [24]:
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

for k in range(1, len(X_train_encoded.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train_scaled, y_train)
    X_test_selected = selector.transform(X_test_scaled)
    
    model = RidgeCV()
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

NameError: ignored

In [21]:
# Correlation matrix!
X_train.corr()

BOROUGH_3  ...  TAX_CLASS_AT_TIME_OF_SALE
BOROUGH_3                              1.000000  ...                   0.052815
BOROUGH_4                             -0.487370  ...                  -0.081014
BOROUGH_2                             -0.232983  ...                  -0.000621
BOROUGH_5                             -0.278943  ...                  -0.098055
BOROUGH_1                             -0.128135  ...                   0.265151
NEIGHBORHOOD_OTHER                    -0.079637  ...                  -0.142022
NEIGHBORHOOD_BEDFORD STUYVESANT        0.239398  ...                   0.057822
NEIGHBORHOOD_FLUSHING-NORTH           -0.108207  ...                   0.085306
NEIGHBORHOOD_UPPER WEST SIDE (79-96)  -0.024338  ...                   0.045305
NEIGHBORHOOD_UPPER WEST SIDE (59-79)  -0.024338  ...                   0.045305
NEIGHBORHOOD_BOROUGH PARK              0.196605  ...                   0.023292
NEIGHBORHOOD_ASTORIA                  -0.069658  ...                   0.041073
NEIGHBORHOOD_FOREST HILLS             -0.037746  ...                  -0.008773
NEIGHBORHOOD_GRAMERCY                 -0.036948  ...                   0.068779
NEIGHBORHOOD_UPPER EAST SIDE (59-79)  -0.018846  ...                   0.035082
NEIGHBORHOOD_UPPER EAST SIDE (79-96)  -0.028806  ...                   0.053622
TAX_CLASS_AT_PRESENT_1                -0.076503  ...                  -0.748420
TAX_CLASS_AT_PRESENT_4                 0.012190  ...                   0.841849
TAX_CLASS_AT_PRESENT_2                -0.047386  ...                   0.351969
TAX_CLASS_AT_PRESENT_2A                0.073641  ...                   0.211484
TAX_CLASS_AT_PRESENT_1B               -0.003545  ...                  -0.049443
TAX_CLASS_AT_PRESENT_1A               -0.001032  ...                  -0.059259
TAX_CLASS_AT_PRESENT_2C                0.184453  ...                   0.165562
TAX_CLASS_AT_PRESENT_2B                0.015978  ...                   0.068779
TAX_CLASS_AT_PRESENT_1C                0.066035  ...                  -0.018075
BLOCK                                 -0.099426  ...                  -0.205822
LOT                                    0.031224  ...                   0.371078
ZIP_CODE                               0.127584  ...                  -0.111755
RESIDENTIAL_UNITS                      0.048967  ...                   0.062452
COMMERCIAL_UNITS                       0.072543  ...                   0.248000
TOTAL_UNITS                            0.088357  ...                   0.146994
LAND_SQUARE_FEET                      -0.079994  ...                   0.071203
GROSS_SQUARE_FEET                      0.054696  ...                   0.147352
YEAR_BUILT                            -0.018596  ...                  -0.146448
TAX_CLASS_AT_TIME_OF_SALE              0.052815  ...                   1.000000

[35 rows x 35 columns]

In [26]:
# Now with regularization via ridge regression
from sklearn.linear_model import Ridge

ridge_reg = Ridge().fit(X, y)
mean_squared_error(y, ridge_reg.predict(X))

NameError: ignored

In [29]:
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train_scaled, y_train)
    X_test_selected = selector.transform(X_test_scaled)
    
    model = RidgeCV()
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

1 features
Test MAE: $286,108 

2 features
Test MAE: $272,382 

3 features
Test MAE: $318,791 

4 features
Test MAE: $297,424 

5 features
Test MAE: $304,440 

6 features
Test MAE: $302,010 

7 features
Test MAE: $305,806 

8 features
Test MAE: $317,192 

9 features
Test MAE: $376,081 

10 features
Test MAE: $379,632 

11 features
Test MAE: $395,728 

12 features
Test MAE: $396,215 

13 features
Test MAE: $388,282 

14 features
Test MAE: $429,284 

15 features
Test MAE: $433,884 

16 features
Test MAE: $434,581 

17 features
Test MAE: $375,285 

18 features
Test MAE: $383,102 

19 features
Test MAE: $382,126 

20 features
Test MAE: $382,197 

21 features
Test MAE: $382,386 

22 features
Test MAE: $420,834 

23 features
Test MAE: $419,971 

24 features
Test MAE: $420,817 

25 features
Test MAE: $420,778 

26 features
Test MAE: $421,190 

27 features
Test MAE: $421,200 

28 features
Test MAE: $420,656 

29 features
Test MAE: $420,790 

30 features
Test MAE: $404,191 

31 features
Test MA

In [31]:
k = 27
selector = SelectKBest(score_func=f_regression, k=k)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)

all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\nFeatures not selected:')
for name in unselected_names:
    print(name)

Features selected:
BOROUGH_3
BOROUGH_2
BOROUGH_5
BOROUGH_1
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_UPPER WEST SIDE (59-79)
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_GRAMERCY
NEIGHBORHOOD_UPPER EAST SIDE (79-96)
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_4
TAX_CLASS_AT_PRESENT_2
TAX_CLASS_AT_PRESENT_2A
TAX_CLASS_AT_PRESENT_1A
TAX_CLASS_AT_PRESENT_2C
TAX_CLASS_AT_PRESENT_2B
TAX_CLASS_AT_PRESENT_1C
BLOCK
ZIP_CODE
RESIDENTIAL_UNITS
LAND_SQUARE_FEET
GROSS_SQUARE_FEET
TAX_CLASS_AT_TIME_OF_SALE

Features not selected:
BOROUGH_4
NEIGHBORHOOD_UPPER WEST SIDE (79-96)
NEIGHBORHOOD_UPPER EAST SIDE (59-79)
TAX_CLASS_AT_PRESENT_1B
LOT
COMMERCIAL_UNITS
TOTAL_UNITS
YEAR_BUILT
